In [1]:
import json

with open("../data/customers/sarah.json") as f:
    sarah = json.load(f)

for month, purchases in sarah["purchase_history"].items():
    categories = [p["category"] for p in purchases]
    print(month, categories)


month_1 ['Alcohol', 'Luxury', 'Raw_Food', 'Caffeine']
month_2 ['Alcohol', 'Luxury', 'Alcohol']
month_3 ['Comfort', 'Comfort', 'Health', 'Staples', 'Health']


In [2]:
from collections import Counter

def analyze_behavior_shift(purchase_history):
    """
    Compares baseline (month_1 + month_2) vs recent (month_3)
    and returns behavioral change signals.
    """
    # Combine baseline months
    baseline_categories = []
    for month in ["month_1", "month_2"]:
        baseline_categories.extend(
            [item["category"] for item in purchase_history[month]]
        )

    recent_categories = [
        item["category"] for item in purchase_history["month_3"]
    ]

    baseline_count = Counter(baseline_categories)
    recent_count = Counter(recent_categories)

    disappeared = [
        cat for cat in baseline_count
        if cat not in recent_count
    ]

    emerged = [
        cat for cat in recent_count
        if cat not in baseline_count
    ]

    signals = {
        "disappeared_categories": disappeared,
        "emerged_categories": emerged,
        "baseline_distribution": dict(baseline_count),
        "recent_distribution": dict(recent_count)
    }

    return signals


In [5]:
signals = analyze_behavior_shift(sarah["purchase_history"])
signals

{'disappeared_categories': ['Alcohol', 'Luxury', 'Raw_Food', 'Caffeine'],
 'emerged_categories': ['Comfort', 'Health', 'Staples'],
 'baseline_distribution': {'Alcohol': 3,
  'Luxury': 2,
  'Raw_Food': 1,
  'Caffeine': 1},
 'recent_distribution': {'Comfort': 2, 'Health': 2, 'Staples': 1}}

In [6]:
# Prepare input for Agent 2 (Life-Event Reasoning)

behavior_signals = {
    "disappeared_categories": signals["disappeared_categories"],
    "emerged_categories": signals["emerged_categories"],
    "baseline_distribution": signals["baseline_distribution"],
    "recent_distribution": signals["recent_distribution"]
}

feedback_text = " ".join(
    f["text"] for f in sarah["feedback"]
)

behavior_signals, feedback_text


({'disappeared_categories': ['Alcohol', 'Luxury', 'Raw_Food', 'Caffeine'],
  'emerged_categories': ['Comfort', 'Health', 'Staples'],
  'baseline_distribution': {'Alcohol': 3,
   'Luxury': 2,
   'Raw_Food': 1,
   'Caffeine': 1},
  'recent_distribution': {'Comfort': 2, 'Health': 2, 'Staples': 1}},
 'Carrying grocery bags has become really hard lately. I wish you offered home delivery for loyal customers.')

In [7]:
life_event_prompt = f"""
You are a senior retail behavioral analyst for a supermarket loyalty platform.

You are given:
1. Behavioral change signals derived from purchase history
2. Customer feedback text

Your task:
- Infer the MOST LIKELY life-event or lifestyle shift
- Base your reasoning ONLY on the provided signals
- Avoid speculation beyond evidence
- Explain causality clearly in business terms

Behavioral Signals:
{behavior_signals}

Customer Feedback:
\"\"\"
{feedback_text}
\"\"\"

Return a STRICT JSON object with this schema:
{{
  "life_event": "<short label>",
  "confidence": "<High | Medium | Low>",
  "evidence": ["list", "of", "supporting signals"],
  "business_risk": "<what happens if we do nothing>"
}}

Do not return any extra text.
"""


In [17]:
from groq import Groq
print("Groq imported successfully")

Groq imported successfully


In [ ]:
import os
assert os.getenv("GROQ_API_KEY"), "Set GROQ_API_KEY in environment"

In [19]:
from groq import Groq
import json
import os

client = Groq(api_key=os.environ["GROQ_API_KEY"])

def run_groq(prompt, model="llama-3.3-70b-versatile"):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt}
        ],
        temperature=0.2
    )
    return response.choices[0].message.content.strip()

raw_output = run_groq(life_event_prompt)
raw_output


'```\n{\n  "life_event": "New Parent or Health Issue",\n  "confidence": "High",\n  "evidence": [\n    "disappearance of Alcohol, Luxury, and Caffeine categories",\n    "emergence of Comfort, Health, and Staples categories",\n    "customer feedback mentioning difficulty carrying grocery bags"\n  ],\n  "business_risk": "Loss of customer loyalty and potential churn if home delivery option is not provided"\n}\n```'

In [22]:
def clean_llm_json(text: str) -> str:
    """
    Removes markdown code fences from LLM outputs.
    """
    text = text.strip()
    if text.startswith("```"):
        text = text.replace("```", "")
    return text.strip()


In [24]:
clean_output = clean_llm_json(raw_output)

try:
    diagnosis = json.loads(clean_output)
    diagnosis
except json.JSONDecodeError as e:
    print("❌ Still invalid JSON")
    print(clean_output)

In [25]:
agent_2_output = diagnosis
agent_2_output

{'life_event': 'New Parent or Health Issue',
 'confidence': 'High',
 'evidence': ['disappearance of Alcohol, Luxury, and Caffeine categories',
  'emergence of Comfort, Health, and Staples categories',
  'customer feedback mentioning difficulty carrying grocery bags'],
 'business_risk': 'Loss of customer loyalty and potential churn if home delivery option is not provided'}

In [26]:
loyalty_strategy_prompt = f"""
You are a senior loyalty strategist designing interventions using Antavo's Enterprise Loyalty Cloud.

You are given a customer diagnosis derived from behavioral and sentiment analysis.

Your task:
- Prevent churn
- Increase long-term loyalty (CLV)
- Use Antavo-native concepts (Segments, Clubs, Rewards, Channels)
- Be empathetic and brand-safe
- Avoid discounts unless absolutely necessary

Customer Diagnosis:
{agent_2_output}

Return a STRICT JSON object with this schema:
{{
  "segment": "<segment name>",
  "club_action": {{
    "action": "Invite | Remove | None",
    "club_name": "<club name>"
  }},
  "rewards": [
    {{
      "type": "<Service | Points | Access>",
      "description": "<what the reward is>",
      "duration": "<timeframe>",
      "business_rationale": "<why this helps retention>"
    }}
  ],
  "channels": ["Email", "Push", "SMS"],
  "message_tone": "<Supportive | Apologetic | Encouraging>",
  "example_message": "<short customer-facing message>",
  "guardrails": "<what should NOT be promoted>"
}}

Do not include markdown or extra text. Only return JSON.
"""


In [27]:
raw_strategy_output = run_groq(loyalty_strategy_prompt)
raw_strategy_output


'{\n  "segment": "New Parents or Health-Conscious Customers",\n  "club_action": {\n    "action": "Invite",\n    "club_name": "Convenience Club"\n  },\n  "rewards": [\n    {\n      "type": "Service",\n      "description": "Free home delivery for 3 months",\n      "duration": "90 days",\n      "business_rationale": "Helps alleviate difficulties with carrying grocery bags and increases loyalty by providing a convenient service"\n    },\n    {\n      "type": "Access",\n      "description": "Priority access to health and wellness products",\n      "duration": "Ongoing",\n      "business_rationale": "Supports customers\' new health-focused purchasing behavior and encourages long-term loyalty"\n    }\n  ],\n  "channels": ["Email", "Push"],\n  "message_tone": "Supportive",\n  "example_message": "We\'re here to support you during this time. Enjoy free home delivery and priority access to healthy products with our Convenience Club.",\n  "guardrails": "Alcohol, Luxury, and Caffeine categories"\n}

In [28]:
clean_strategy_output = clean_llm_json(raw_strategy_output)

try:
    loyalty_strategy = json.loads(clean_strategy_output)
    loyalty_strategy
except json.JSONDecodeError:
    print("❌ Invalid JSON from Agent 3")
    print(clean_strategy_output)


In [29]:
loyalty_strategy

{'segment': 'New Parents or Health-Conscious Customers',
 'club_action': {'action': 'Invite', 'club_name': 'Convenience Club'},
 'rewards': [{'type': 'Service',
   'description': 'Free home delivery for 3 months',
   'duration': '90 days',
   'business_rationale': 'Helps alleviate difficulties with carrying grocery bags and increases loyalty by providing a convenient service'},
  {'type': 'Access',
   'description': 'Priority access to health and wellness products',
   'duration': 'Ongoing',
   'business_rationale': "Supports customers' new health-focused purchasing behavior and encourages long-term loyalty"}],
 'channels': ['Email', 'Push'],
 'message_tone': 'Supportive',
 'example_message': "We're here to support you during this time. Enjoy free home delivery and priority access to healthy products with our Convenience Club.",
 'guardrails': 'Alcohol, Luxury, and Caffeine categories'}

{'disappeared_categories': ['Alcohol', 'Luxury', 'Raw_Food', 'Caffeine'],
 'emerged_categories': ['Comfort', 'Health', 'Staples'],
 'baseline_distribution': {'Alcohol': 3,
  'Luxury': 2,
  'Raw_Food': 1,
  'Caffeine': 1},
 'recent_distribution': {'Comfort': 2, 'Health': 2, 'Staples': 1}}